2022/3/22 </br>
https://github.com/pytorch/vision/blob/main/torchvision/models/vgg.py

In [2]:
import torch
import torch.nn as nn
from typing import Union, List, Dict, Any, Optional, cast

In [ ]:
__all__ = [
    "VGG",
    "vgg11",
    "vgg11_bn",  # vgg11 batch normalization
    "vgg13",
    "vgg13_bn",
    "vgg16",
    "vgg16_bn",
    "vgg19",
    "vgg19_bn",
]

In [ ]:
class VGG(nn.Module):
    def __init__(
        self, 
        features: nn.Module, 
        num_classes: int = 1000, 
        init_weights: bool = True, 
        dropout: float = 0.5
    ):
        super().__init__()
        
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x: torch.Tensor):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
def _vgg(  # Generate VGG model
    cfg: str, 
    batch_norm: bool, 
    weights: Optional[WeightsEnum], 
    progress: bool, 
    **kwargs: Any
    ) -> VGG:
    
    if weights is not None:
        kwargs["init_weights"] = False
        if weights.meta["categories"] is not None:
            _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))
            
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))
    return model

In [ ]:
def make_layers(
    cfg: List[Union[str, int]], 
    batch_norm: bool = False
) -> nn.Sequential:
    
    layers: List[nn.Module] = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            v = cast(int, v)
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [3]:
cfgs: Dict[str, List[Union[str, int]]] = {
    "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],
}

In [ ]:
def vgg11(
    pretrained: bool = False, 
    progress: bool = True, 
    **kwargs: Any
) -> VGG:
    return _vgg("A", False, weights, progress, **kwargs)